In [1]:
using ITensors
using LinearAlgebra
using Plots
using Statistics
using CSV
using DataFrames

In [3]:
function dmrg_ZXX(a, L)
    sites = siteinds("Fermion", L; conserve_qns=false)

    os = OpSum()
    
    # H_CTI
    for j=1:L
        jp1 = j % L + 1  # j+1 with periodic boundary
        jm1 = j == 1 ? L : j - 1  # j-1 with periodic boundary

        coef =a*(1-a)  # 係数の計算
        os += coef, "C", jp1, "C", j
        os += -coef, "Cdag", jp1, "Cdag", j
        os += coef, "Cdag", jp1, "C", j
        os += -coef, "C", jp1, "Cdag", j
        os += -2*coef, "N", j
    end
    
   # H_int
   for j=1:L
        jp1 = j % L + 1  # j+1 with periodic boundary
        jm1 = j == 1 ? L : j - 1  # j-1 with periodic boundary

        coef = a^2  # 係数の計算
        os += coef, "C", jp1, "C", j
        os += -coef, "Cdag", jp1, "Cdag", j
        os += coef, "Cdag", jp1, "C", j
        os += -coef, "C", jp1, "Cdag", j
        os += -2*coef, "N", jm1,"C", jp1, "C", j
        os += 2*coef, "N", jm1, "Cdag", jp1, "Cdag", j
        os += -2*coef, "N", jm1, "Cdag", jp1, "C", j
        os += 2*coef, "N", jm1, "C", jp1, "Cdag", j
    end

    H = MPO(os, sites)
    psi0 = randomMPS(sites)

    # Numerical accuracy parameters
    nsweeps = 100
    maxdim = [10, 20, 100, 100, 200, 300]
    cutoff = 1E-10

    # Find the ground state
    energy_gs, psi_gs = dmrg(H, psi0; nsweeps, maxdim, cutoff,outputlevel=0)

    return energy_gs + (a^2+2*a*(1-a)+(1-a)^2)*L/2 # 修正された部分
end

dmrg_ZXX (generic function with 1 method)

In [6]:
# 各aの値に対して(L, energydens)のペアを保存するための配列
energydens_pairs = Dict()

for a in 0.05:0.05:0.45
    L_values = [8,10,12,14,16,18,20,22,24,26,28,30,32,34,36,38,40,42,44,46,48,50]
    pairs = []

    for L in L_values
        energydens = dmrg_ZXX(a, L) / L
        push!(pairs, (L, energydens))
        println("a=$a, L = $L, Energydens = $energydens")
    end

    # 各aの値に対する(L, energydens)ペアの配列を格納
    energydens_pairs[a] = pairs

    # グラフのプロット
    gr(size=(1000, 300))
    p1 = plot(L_values, [pair[2] for pair in pairs], title="Energydens vs L (Linear) at a=$a", xlabel="L", ylabel="Energydens", legend=false)
    p2 = plot(L_values, [pair[2] for pair in pairs], title="Energydens vs L (Semilog y) at a=$a", xlabel="L", ylabel="Energydens", legend=false, yscale=:log10)
    p3 = plot(L_values, [pair[2] for pair in pairs], title="Energydens vs L (Loglog) at a=$a", xlabel="L", ylabel="Energydens", legend=false, xscale=:log10, yscale=:log10)
    combined_plot = plot(p1, p2, p3, layout=(1, 3))

    # PDFとして保存
    filename = "energydens_vs_L_a$(a).pdf"
    savefig(combined_plot, filename)
end

# (L, energydens)ペアをファイルに保存
open("energydens_pairs.txt", "w") do file
    for (a, pairs) in energydens_pairs
        write(file, "a=$a: ")
        for pair in pairs
            write(file, "($(pair[1]), $(pair[2])), ")
        end
        write(file, "\n")
    end
end

a=0.05, L = 8, Energydens = 0.39369425678333014
a=0.05, L = 10, Energydens = 0.393414044483992
a=0.05, L = 12, Energydens = 0.3932619234017154
a=0.05, L = 14, Energydens = 0.39317023273094903
a=0.05, L = 16, Energydens = 0.3931107358851802
a=0.05, L = 18, Energydens = 0.39306995149247576
a=0.05, L = 20, Energydens = 0.39304078190929126
a=0.05, L = 22, Energydens = 0.3930192014831213
a=0.05, L = 24, Energydens = 0.39300278880258177
a=0.05, L = 26, Energydens = 0.3929900164850298
a=0.05, L = 28, Energydens = 0.392979882425725
a=0.05, L = 30, Energydens = 0.39297170701884004
a=0.05, L = 32, Energydens = 0.3929650162230362
a=0.05, L = 34, Energydens = 0.3929594711502541
a=0.05, L = 36, Energydens = 0.3929548244154911
a=0.05, L = 38, Energydens = 0.3929508919352014
a=0.05, L = 40, Energydens = 0.3929475344911885
a=0.05, L = 42, Energydens = 0.3929446451920113
a=0.05, L = 44, Energydens = 0.39294214088334345
a=0.05, L = 46, Energydens = 0.39293995608354015
a=0.05, L = 48, Energydens = 0.3929

In [7]:
# 各aの値に対して(L, energydens)のペアを保存するための配列
energydens_pairs = Dict()

for a in 0.455:0.005:0.48
    L_values = [8,10,12,14,16,18,20,22,24,26,28,30,32,34,36,38,40,42,44,46,48,50]
    pairs = []

    for L in L_values
        energydens = dmrg_ZXX(a, L) / L
        push!(pairs, (L, energydens))
        println("a=$a, L = $L, Energydens = $energydens")
    end

    # 各aの値に対する(L, energydens)ペアの配列を格納
    energydens_pairs[a] = pairs

    # グラフのプロット
    gr(size=(1000, 300))
    p1 = plot(L_values, [pair[2] for pair in pairs], title="Energydens vs L (Linear) at a=$a", xlabel="L", ylabel="Energydens", legend=false)
    p2 = plot(L_values, [pair[2] for pair in pairs], title="Energydens vs L (Semilog y) at a=$a", xlabel="L", ylabel="Energydens", legend=false, yscale=:log10)
    p3 = plot(L_values, [pair[2] for pair in pairs], title="Energydens vs L (Loglog) at a=$a", xlabel="L", ylabel="Energydens", legend=false, xscale=:log10, yscale=:log10)
    combined_plot = plot(p1, p2, p3, layout=(1, 3))

    # PDFとして保存
    filename = "energydens_vs_L_a$(a).pdf"
    savefig(combined_plot, filename)
end

# (L, energydens)ペアをファイルに保存
open("energydens_pairs.txt", "w") do file
    for (a, pairs) in energydens_pairs
        write(file, "a=$a: ")
        for pair in pairs
            write(file, "($(pair[1]), $(pair[2])), ")
        end
        write(file, "\n")
    end
end

a=0.455, L = 8, Energydens = 0.0009203829281062004
a=0.455, L = 10, Energydens = 0.0006603724640010355
a=0.455, L = 12, Energydens = 0.0005052690790154912
a=0.455, L = 14, Energydens = 0.0004050414959858425
a=0.455, L = 16, Energydens = 0.0003361812680386933
a=0.455, L = 18, Energydens = 0.0002865801454763586
a=0.455, L = 20, Energydens = 0.00024949568909944376
a=0.455, L = 22, Energydens = 0.0002209261043592486
a=0.455, L = 24, Energydens = 0.0001983692846434515
a=0.455, L = 26, Energydens = 0.00018019180490790577
a=0.455, L = 28, Energydens = 0.00016528818595488
a=0.455, L = 30, Energydens = 0.0001528869927627241
a=0.455, L = 32, Energydens = 0.00014243558215332186
a=0.455, L = 34, Energydens = 0.00013352888746521284
a=0.455, L = 36, Energydens = 0.0001258640223462177
a=0.455, L = 38, Energydens = 0.00011921055508844422
a=0.455, L = 40, Energydens = 0.00011339032190678111
a=0.455, L = 42, Energydens = 0.00010826367952202329
a=0.455, L = 44, Energydens = 0.00010371979237756467
a=0.455

In [14]:
function dmrg_gap(a, L)
    sites = siteinds("Fermion", L; conserve_qns=false)

    os = OpSum()
    
    # H_CTI
    for j=1:L
        jp1 = j % L + 1  # j+1 with periodic boundary
        jm1 = j == 1 ? L : j - 1  # j-1 with periodic boundary

        coef =a*(1-a)  # 係数の計算
        os += coef, "C", jp1, "C", j
        os += -coef, "Cdag", jp1, "Cdag", j
        os += coef, "Cdag", jp1, "C", j
        os += -coef, "C", jp1, "Cdag", j
        os += -2*coef, "N", j
    end
    
   # H_int
   for j=1:L
        jp1 = j % L + 1  # j+1 with periodic boundary
        jm1 = j == 1 ? L : j - 1  # j-1 with periodic boundary

        coef = a^2  # 係数の計算
        os += coef, "C", jp1, "C", j
        os += -coef, "Cdag", jp1, "Cdag", j
        os += coef, "Cdag", jp1, "C", j
        os += -coef, "C", jp1, "Cdag", j
        os += -2*coef, "N", jm1,"C", jp1, "C", j
        os += 2*coef, "N", jm1, "Cdag", jp1, "Cdag", j
        os += -2*coef, "N", jm1, "Cdag", jp1, "C", j
        os += 2*coef, "N", jm1, "C", jp1, "Cdag", j
    end

    H = MPO(os, sites)
    psi0 = randomMPS(sites)

    # Numerical accuracy parameters
    nsweeps = 100
    maxdim = [10, 20, 100, 100, 200, 300]
    cutoff = 1E-10

    # Find the ground state
    energy_gs, psi_gs = dmrg(H, psi0; nsweeps, maxdim, cutoff,outputlevel=0)

    # Find the first excited state in the orthogonal complement of the ground state
    energy_es1, psi_es1 = dmrg(H,[psi_gs],psi0; nsweeps, maxdim, cutoff,outputlevel=0)

    # Find the second excited state in the orthogonal complement of the first two states
    energy_es2, psi_es2 = dmrg(H,[psi_gs,psi_es1],psi0; nsweeps, maxdim, cutoff,outputlevel=0)
    
    gap=energy_es2 - energy_gs

    return gap
end

dmrg_gap (generic function with 1 method)

In [6]:
# 各aの値に対して(L, gap)のペアを保存するための配列
gap_pairs = Dict()

for a in 0.05:0.05:0.45
    L_values = [8,10,12,14,16,18,20,22,24,26,28,30,32,34,36,38,40,42,44,46,48,50]
    pairs = []

    for L in L_values
        gap = dmrg_gap(a, L)
        push!(pairs, (L, gap))
        println("a=$a, L = $L, Gap = $gap")
    end

    # 各aの値に対する(L, gap)ペアの配列を格納
    gap_pairs[a] = pairs

    # グラフのプロット
    gr(size=(1000, 300))
    p1 = plot(L_values, [pair[2] for pair in pairs], title="Gap vs L (Linear) at a=$a", xlabel="L", ylabel="Gap", legend=false)
    p2 = plot(L_values, [pair[2] for pair in pairs], title="Gap vs L (Semilog y) at a=$a", xlabel="L", ylabel="Gap", legend=false, yscale=:log10)
    p3 = plot(L_values, [pair[2] for pair in pairs], title="Gap vs L (Loglog) at a=$a", xlabel="L", ylabel="Gap", legend=false, xscale=:log10, yscale=:log10)
    combined_plot = plot(p1, p2, p3, layout=(1, 3))

    # PDFとして保存
    filename = "gap_vs_L_a$(a).pdf"
    savefig(combined_plot, filename)
end

# (L, gap)ペアをファイルに保存
open("gap_pairs.txt", "w") do file
    for (a, pairs) in gap_pairs
        write(file, "a=$a: ")
        for pair in pairs
            write(file, "($(pair[1]), $(pair[2])), ")
        end
        write(file, "\n")
    end
end

# 各aの値に対して(L, gap)のペアを保存するための配列
gap_pairs = Dict()

for a in 0.455:0.005:0.5
    L_values = [8,10,12,14,16,18,20,22,24,26,28,30,32,34,36,38,40,42,44,46,48,50]
    pairs = []

    for L in L_values
        gap = dmrg_gap(a, L)
        push!(pairs, (L, gap))
        println("a=$a, L = $L, Gap = $gap")
    end

    # 各aの値に対する(L, gap)ペアの配列を格納
    gap_pairs[a] = pairs

    # グラフのプロット
    gr(size=(1000, 300))
    p1 = plot(L_values, [pair[2] for pair in pairs], title="Gap vs L (Linear) at a=$a", xlabel="L", ylabel="Gap", legend=false)
    p2 = plot(L_values, [pair[2] for pair in pairs], title="Gap vs L (Semilog y) at a=$a", xlabel="L", ylabel="Gap", legend=false, yscale=:log10)
    p3 = plot(L_values, [pair[2] for pair in pairs], title="Gap vs L (Loglog) at a=$a", xlabel="L", ylabel="Gap", legend=false, xscale=:log10, yscale=:log10)
    combined_plot = plot(p1, p2, p3, layout=(1, 3))

    # PDFとして保存
    filename = "gap_vs_L_a$(a).pdf"
    savefig(combined_plot, filename)
end

# (L, gap)ペアをファイルに保存
open("gap_pairs2.txt", "w") do file
    for (a, pairs) in gap_pairs
        write(file, "a=$a: ")
        for pair in pairs
            write(file, "($(pair[1]), $(pair[2])), ")
        end
        write(file, "\n")
    end
end

a=0.05, L = 8, Gap = 0.07197981765238948
a=0.05, L = 10, Gap = 0.05830937590833174
a=0.05, L = 12, Gap = 0.0489282174098602
a=0.05, L = 14, Gap = 0.042115366734124304
a=0.05, L = 16, Gap = 0.036952390240364785
a=0.05, L = 18, Gap = 0.032908799999743055
a=0.05, L = 20, Gap = 0.029658168828032405
a=0.05, L = 22, Gap = 0.026989136632874278
a=0.05, L = 24, Gap = 0.024759029801360022
a=0.05, L = 26, Gap = 0.022868156693645947
a=0.05, L = 28, Gap = 0.02124480296856035
a=0.05, L = 30, Gap = 0.019836086577480483
a=0.05, L = 32, Gap = 0.018602171211676843
a=0.05, L = 34, Gap = 0.017512485551980372
a=0.05, L = 36, Gap = 0.01654318069223626
a=0.05, L = 38, Gap = 0.015675383917536756
a=0.05, L = 40, Gap = 0.014893965597199355
a=0.05, L = 42, Gap = 0.014186656622171334
a=0.05, L = 44, Gap = 0.013543403994535907
a=0.05, L = 46, Gap = 0.012955891925672702
a=0.05, L = 48, Gap = 0.012417183940456766
a=0.05, L = 50, Gap = 0.011921445584747836
a=0.1, L = 8, Gap = 0.13459060926151256
a=0.1, L = 10, Gap = 

a=0.455, L = 8, Gap = 0.2318643353470904
a=0.455, L = 10, Gap = 0.19722100038895007
a=0.455, L = 12, Gap = 0.17144669003809643
a=0.455, L = 14, Gap = 0.14866807679882665


LoadError: InterruptException:

In [8]:
function dmrg_APBC_gap(a, L)
    sites = siteinds("Fermion", L; conserve_qns=false)

    os = OpSum()
    
    # H_CTI
    for j=1:L-1
        jp1 = j % L + 1  # j+1 with periodic boundary

        coef =a*(1-a)  # 係数の計算
        os += coef, "C", jp1, "C", j
        os += -coef, "Cdag", jp1, "Cdag", j
        os += coef, "Cdag", jp1, "C", j
        os += -coef, "C", jp1, "Cdag", j
        os += -2*coef, "N", j
    end
    
    coef =a*(1-a)  # 係数の計算
    os += -coef, "C", 1, "C", L
    os += coef, "Cdag", 1, "Cdag", L
    os += -coef, "Cdag", 1, "C", L
    os += coef, "C", 1, "Cdag", L
    os += -2*coef, "N", L
    
   # H_int
   for j=1:L-1
        jp1 = j % L + 1  # j+1 with periodic boundary
        jm1 = j == 1 ? L : j - 1  # j-1 with periodic boundary

        coef = a^2  # 係数の計算
        os += coef, "C", jp1, "C", j
        os += -coef, "Cdag", jp1, "Cdag", j
        os += coef, "Cdag", jp1, "C", j
        os += -coef, "C", jp1, "Cdag", j
        os += -2*coef, "N", jm1,"C", jp1, "C", j
        os += 2*coef, "N", jm1, "Cdag", jp1, "Cdag", j
        os += -2*coef, "N", jm1, "Cdag", jp1, "C", j
        os += 2*coef, "N", jm1, "C", jp1, "Cdag", j
    end
    
        coef = a^2  # 係数の計算
        os += -coef, "C", 1, "C", L
        os += coef, "Cdag", 1, "Cdag", L
        os += -coef, "Cdag", 1, "C", L
        os += coef, "C", 1, "Cdag", L
        os += 2*coef, "N", L-1,"C", 1, "C", L
        os += -2*coef, "N", L-1, "Cdag", 1, "Cdag", L
        os += 2*coef, "N", L-1, "Cdag", 1, "C", L
        os += -2*coef, "N", L-1, "C", 1, "Cdag", L

    H = MPO(os, sites)
    psi0 = randomMPS(sites)

    # Numerical accuracy parameters
    nsweeps = 100
    maxdim = [10, 20, 100, 100, 200, 300]
    cutoff = 1E-10

    # Find the ground state
    energy_gs, psi_gs = dmrg(H, psi0; nsweeps, maxdim, cutoff,outputlevel=0)

    # Find the first excited state in the orthogonal complement of the ground state
    energy_es1, psi_es1 = dmrg(H,[psi_gs],psi0; nsweeps, maxdim, cutoff,outputlevel=0)
    
    gap=energy_es1 - energy_gs

    return gap
end

dmrg_APBC_gap (generic function with 1 method)

In [12]:
function calculate_gaps(a_values)
    gap_pairs = Dict()
    for a in a_values
        println("Calculating for a = $a...")
        gap = dmrg_APBC_gap(a, 50)
        gap_pairs[a] = gap
    end
    return gap_pairs
end

function save_gap_pairs_to_file(gap_pairs, filename)
    open(filename, "w") do file
        for (a, gap) in gap_pairs
            write(file, "($a, $gap)\n")
        end
    end
end

save_gap_pairs_to_file (generic function with 1 method)

In [13]:
# aの値の範囲を設定
all_a_values = collect(0.05:0.05:0.45)
append!(all_a_values, 0.455:0.005:0.5)

# エネルギーギャップの計算
gap_pairs = calculate_gaps(all_a_values)

# 結果をファイルに保存
save_gap_pairs_to_file(gap_pairs, "Agap_pairs_all.txt")

Calculating for a = 0.05...
Calculating for a = 0.1...
Calculating for a = 0.15...
Calculating for a = 0.2...
Calculating for a = 0.25...
Calculating for a = 0.3...
Calculating for a = 0.35...
Calculating for a = 0.4...
Calculating for a = 0.45...
Calculating for a = 0.455...
Calculating for a = 0.46...
Calculating for a = 0.465...
Calculating for a = 0.47...
Calculating for a = 0.475...
Calculating for a = 0.48...
Calculating for a = 0.485...
Calculating for a = 0.49...
Calculating for a = 0.495...
Calculating for a = 0.5...


In [15]:
# 各aの値に対して(L, gap)のペアを保存するための配列
gap_pairs = Dict()

for a in 0.455:0.005:0.5
    L_values = [8,10,12,14,16,18,20,22,24,26,28,30,32,34,36,38,40,42,44,46,48,50]
    pairs = []

    for L in L_values
        gap = dmrg_gap(a, L)
        push!(pairs, (L, gap))
        println("a=$a, L = $L, Gap = $gap")
    end

    # 各aの値に対する(L, gap)ペアの配列を格納
    gap_pairs[a] = pairs

    # グラフのプロット
    gr(size=(1000, 300))
    p1 = plot(L_values, [pair[2] for pair in pairs], title="Gap vs L (Linear) at a=$a", xlabel="L", ylabel="Gap", legend=false)
    p2 = plot(L_values, [pair[2] for pair in pairs], title="Gap vs L (Semilog y) at a=$a", xlabel="L", ylabel="Gap", legend=false, yscale=:log10)
    p3 = plot(L_values, [pair[2] for pair in pairs], title="Gap vs L (Loglog) at a=$a", xlabel="L", ylabel="Gap", legend=false, xscale=:log10, yscale=:log10)
    combined_plot = plot(p1, p2, p3, layout=(1, 3))

    # PDFとして保存
    filename = "gap_vs_L_a$(a).pdf"
    savefig(combined_plot, filename)
end

# (L, gap)ペアをファイルに保存
open("gap_pairs2.txt", "w") do file
    for (a, pairs) in gap_pairs
        write(file, "a=$a: ")
        for pair in pairs
            write(file, "($(pair[1]), $(pair[2])), ")
        end
        write(file, "\n")
    end
end

a=0.455, L = 8, Gap = 0.23186433534709083
a=0.455, L = 10, Gap = 0.19722100041205604
a=0.455, L = 12, Gap = 0.17144669015105496
a=0.455, L = 14, Gap = 0.14866807686072825
a=0.455, L = 16, Gap = 0.12865332383468076
a=0.455, L = 18, Gap = 0.11353315613305526
a=0.455, L = 20, Gap = 0.10169569447950089
a=0.455, L = 22, Gap = 0.09217073992422087
a=0.455, L = 24, Gap = 0.08433825560113561
a=0.455, L = 26, Gap = 0.07778283941631514
a=0.455, L = 28, Gap = 0.07221538757733903
a=0.455, L = 30, Gap = 0.06742829108172543
a=0.455, L = 32, Gap = 0.06326850037457454
a=0.455, L = 34, Gap = 0.05962064032758718
a=0.455, L = 36, Gap = 0.05639605399731806
a=0.455, L = 38, Gap = 0.05352546590371787
a=0.455, L = 40, Gap = 0.05095394555497634
a=0.455, L = 42, Gap = 0.04863737277483082
a=0.455, L = 44, Gap = 0.04653989044590645
a=0.455, L = 46, Gap = 0.044632058574606503
a=0.455, L = 48, Gap = 0.04288947725219572
a=0.455, L = 50, Gap = 0.041291744782760986
a=0.46, L = 8, Gap = 0.2283373740366912
a=0.46, L = 1

LoadError: InterruptException: